In [ ]:
#installing libraries
!pip install selenium
!pip install scrapy
!pip install crochet
!pip install telegram
!pip install scrapy-rotating-proxies
!pip install scrapy_proxy_pool
!pip install shadow-useragent
!pip install scrapy-useragents
!pip install rotating-free-proxies
!pip install scrapy-user-agents
!pip install scrapy-random-useragent-pro
!pip install fake_headers
!pip install aiohttp
!pip install webdriver_manager

In [ ]:
# Importing Libraries
import csv
import os
import time
import ujson
from random import randint
from typing import Dict, List, Any
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium import webdriver

from selenium.webdriver.chrome.options import Options

In [44]:
# Delete files if present
try:
    os.remove('Authors_URL.txt')
    os.remove('scraper_results.json')
except OSError:
    pass


In [52]:
#Writing Authors to a File:
def write_authors(list1, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for item in list1:
            f.write(item + '\n')

In [53]:
def initCrawlerScraper(seed):
    # Initialize driver for Chrome
    webOpt = webdriver.ChromeOptions()
    webOpt.add_experimental_option('excludeSwitches', ['enable-logging'])
    webOpt.add_argument('--ignore-certificate-errors')
    webOpt.add_argument('--incognito')
    webOpt.add_argument('--headless')  # Updated to use headless mode
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)
    driver.get(seed)  # Start with the original link

    Links = []  # Array with pureportal profiles URL
    pub_data = []  # To store publication information for each pureportal profile

    nextLink = True
    print("Crawler has begun...")
    while nextLink:
        page = driver.page_source
        # XML parser to parse each URL
        bs = BeautifulSoup(page, "lxml")

        # Extracting exact URL by splitting the string into a list
        for link in bs.findAll('a', class_='link person'):
            url = str(link)[str(link).find('https://pureportal.coventry.ac.uk/en/persons/'):].split('"')
            Links.append(url[0])
        # Click on Next button to visit the next page
        try:
            if driver.find_elements(By.CSS_SELECTOR, ".nextLink"):
                element = driver.find_element(By.CSS_SELECTOR, ".nextLink")
                driver.execute_script("arguments[0].click();", element)
            else:
                nextLink = False
        except NoSuchElementException:
            break
    print("Crawler has found", len(Links), "pureportal profiles")
    write_authors(Links, 'Authors_URL.txt')

    print("Scraping publication data for", len(Links), "pureportal profiles...")
    count = 0
    for link in Links:
        # Visit each link to get data
        time.sleep(1)
        driver.get(link)
        try:
            if driver.find_elements(By.CSS_SELECTOR, ".portal_link.btn-primary.btn-large"):
                element = driver.find_elements(By.CSS_SELECTOR, ".portal_link.btn-primary.btn-large")
                for a in element:
                    if "research output".lower() in a.text.lower():
                        driver.execute_script("arguments[0].click();", a)
                        driver.get(driver.current_url)
                        # Get name of Author
                        name = driver.find_element(By.CSS_SELECTOR, "div.header.person-details > h1")
                        r = requests.get(driver.current_url)
                        # Parse all the data via BeautifulSoup
                        soup = BeautifulSoup(r.content, 'lxml')

                        # Extract publication name, publication URL, date, and CU Authors
                        table = soup.find('ul', attrs={'class': 'list-results'})
                        if table is not None:
                            for row in table.findAll('div', attrs={'class': 'result-container'}):
                                data = {}
                                data['name'] = row.h3.a.text
                                data['pub_url'] = row.h3.a['href']
                                date = row.find("span", class_="date")

                                rowitem = row.find_all(['div'])
                                span = row.find_all(['span'])
                                data['cu_author'] = name.text
                                data['date'] = date.text
                                print("Publication Name:", row.h3.a.text)
                                print("Publication URL:", row.h3.a['href'])
                                print("CU Author:", name.text)
                                print("Date:", date.text)
                                print("\n")
                                pub_data.append(data)
            else:
                # Get name of Author
                name = driver.find_element(By.CSS_SELECTOR, "div.header.person-details > h1")
                r = requests.get(link)
                # Parse all the data via BeautifulSoup
                soup = BeautifulSoup(r.content, 'lxml')
                # Extract publication name, publication URL, date, and CU Authors
                table = soup.find('div', attrs={'class': 'relation-list relation-list-publications'})
                if table is not None:
                    for row in table.findAll('div', attrs={'class': 'result-container'}):
                        data = {}
                        data["name"] = row.h3.a.text
                        data['pub_url'] = row.h3.a['href']
                        date = row.find("span", class_="date")
                        rowitem = row.find_all(['div'])
                        span = row.find_all(['span'])
                        data['cu_author'] = name.text
                        data['date'] = date.text
                        print("Publication Name:", row.h3.a.text)
                        print("Publication URL:", row.h3.a['href'])
                        print("CU Author:", name.text)
                        print("Date:", date.text)
                        print("\n")
                        pub_data.append(data)
        except Exception:
            continue

    print("Crawler has scraped data for", len(pub_data), "pureportal publications")
    driver.quit()
    # Writing all the scraped results to a file in JSON format
    with open('scraper_results.json', 'w') as f:
        ujson.dump(pub_data, f)

In [ ]:
#Initializing Crawler and Scrape
initCrawlerScraper('https://pureportal.coventry.ac.uk/en/publications/')

C:\Users\pc\AppData\Local\Temp\ipykernel_6336\3705936045.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=webOpt)


Crawler has begun...
Crawler has found 24915 pureportal profiles
Scraping publication data for 24915 pureportal profiles...
Publication Name: Bletchley Park in Myth and Memory
Publication URL: https://pureportal.coventry.ac.uk/en/publications/bletchley-park-in-myth-and-memory
CU Author: Thomas Knowles
Date: 2024


Publication Name: Book Review: Paradoxa# 31: Climate Fictions. Edited by Alison Sperling
Publication URL: https://pureportal.coventry.ac.uk/en/publications/book-review-paradoxa-31-climate-fictions-edited-by-alison-sperlin
CU Author: Thomas Knowles
Date: 2022


Publication Name: J. G. Ballard and the Sciences
Publication URL: https://pureportal.coventry.ac.uk/en/publications/j-g-ballard-and-the-sciences
CU Author: Thomas Knowles
Date: 30 Jan 2020


Publication Name: J. G. Ballard and Making: An Experiment in Collaborative Practice
Publication URL: https://pureportal.coventry.ac.uk/en/publications/j-g-ballard-and-making-an-experiment-in-collaborative-practice
CU Author: Thomas 

CU Author: Chris Smith
Date: 17 Apr 2014


Publication Name: Operating secret machines: Military women and the intelligence production line, 1939–1945
Publication URL: https://pureportal.coventry.ac.uk/en/publications/operating-secret-machines-military-women-and-the-intelligence-pro
CU Author: Chris Smith
Date: 1 Oct 2014


Publication Name: Review: The Secret Life of Bletchley Park: The History of the Wartime Codebreaking Centre by the Men and Women Who Were There
Publication URL: https://pureportal.coventry.ac.uk/en/publications/review-the-secret-life-of-bletchley-park-the-history-of-the-warti
CU Author: Chris Smith
Date: 24 Feb 2012


Publication Name: 10 Years on with French Arbitration Law Reform: Does the Judicial Control Frustrate or Facilitate the Enforcement of Arbitral Awards?
Publication URL: https://pureportal.coventry.ac.uk/en/publications/10-years-on-with-french-arbitration-law-reform-does-the-judicial-
CU Author: Margaret Liu
Date: 25 Mar 2023


Publication Name: Anti-su

CU Author: Rachael Carew
Date: 17 Dec 2020


Publication Name: Accuracy of computed radiography in osteometry: A comparison of digital imaging techniques and the effect of magnification
Publication URL: https://pureportal.coventry.ac.uk/en/publications/accuracy-of-computed-radiography-in-osteometry-a-comparison-of-di
CU Author: Rachael Carew
Date: Dec 2019


Publication Name: A Preliminary Investigation into the Accuracy of 3D Modeling and 3D Printing in Forensic Anthropology Evidence Reconstruction
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-preliminary-investigation-into-the-accuracy-of-3d-modeling-and-
CU Author: Rachael Carew
Date: Mar 2019


Publication Name: Imaging in forensic science: Five years on
Publication URL: https://pureportal.coventry.ac.uk/en/publications/imaging-in-forensic-science-five-years-on
CU Author: Rachael Carew
Date: Mar 2019


Publication Name: Precision of recumbent crown-heel length when using an infantometer
Publication URL: https

CU Author: Evangelos Gkanas
Date: 27 Feb 2018


Publication Name: A review of a stand-alone Hydrogen system application for private houses in Greece and the use of Phase change materials in their building construction
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-review-of-a-stand-alone-hydrogen-system-application-for-private
CU Author: Evangelos Gkanas
Date: 10 Oct 2017


Publication Name: Experimental and Numerical Study of Low Temperature Methane Steam Reforming for Hydrogen Production
Publication URL: https://pureportal.coventry.ac.uk/en/publications/experimental-and-numerical-study-of-low-temperature-methane-steam
CU Author: Evangelos Gkanas
Date: 25 Dec 2017


Publication Name: Numerical analysis of candidate materials for multi-stage metal hydride hydrogen compression processes
Publication URL: https://pureportal.coventry.ac.uk/en/publications/numerical-analysis-of-candidate-materials-for-multi-stage-metal-h
CU Author: Evangelos Gkanas
Date: Oct 2017


Pub

...

CU Author: Georges Félix
Date: 2018


Publication Name: Puerto Rican EcoFarms after María: Bouncing Back with a Little Help from our Friends
Publication URL: https://pureportal.coventry.ac.uk/en/publications/puerto-rican-ecofarms-after-mar%C3%ADa-bouncing-back-with-a-little-hel
CU Author: Georges Félix
Date: 12 Mar 2018


Publication Name: Ramial wood amendments (Piliostigma reticulatum) mitigate degradation of tropical soils but do not replenish nutrient exports
Publication URL: https://pureportal.coventry.ac.uk/en/publications/ramial-wood-amendments-piliostigma-reticulatum-mitigate-degradati
CU Author: Georges Félix
Date: 25 Aug 2018


Publication Name: Use and management of biodiversity by smallholder farmers in semi-arid West Africa
Publication URL: https://pureportal.coventry.ac.uk/en/publications/use-and-management-of-biodiversity-by-smallholder-farmers-in-semi
CU Author: Georges Félix
Date: Sept 2018


Publication Name: Hurricane María: An Agroecological Turning Point for Puerto

Publication Name: A machine learning based optimisation method to evaluate the crushing behaviours of square tubes with rectangular-hole-type initiators
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-machine-learning-based-optimisation-method-to-evaluate-the-crus
CU Author: Christophe Bastien
Date: 30 Mar 2023


Publication Name: A methodology to investigate and optimise the crashworthiness response of foam-filled twelve right angles thin-walled structures under axial impact
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-methodology-to-investigate-and-optimise-the-crashworthiness-res
CU Author: Christophe Bastien
Date: 15 Apr 2023


Publication Name: A Peak Virtual Power Concept to Compute Brain Injuries Associated with Concussion
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-peak-virtual-power-concept-to-compute-brain-injuries-associated
CU Author: Christophe Bastien
Date: 19 May 2023


Publication Name: Energy Absorpt

Publication Name: Design methodology for crash occupant protection in cabin design of the high speed vessel
Publication URL: https://pureportal.coventry.ac.uk/en/publications/design-methodology-for-crash-occupant-protection-in-cabin-design--2
CU Author: Christophe Bastien
Date: Jan 2017


Publication Name: Enhancing Noise Attenuation in Exhaust Mufflers on Response to Baffle Configuration
Publication URL: https://pureportal.coventry.ac.uk/en/publications/enhancing-noise-attenuation-in-exhaust-mufflers-on-response-to-ba
CU Author: Christophe Bastien
Date: 1 Jul 2017


Publication Name: Investigation of baffle configuration effect on the performance of exhaust mufflers
Publication URL: https://pureportal.coventry.ac.uk/en/publications/investigation-of-baffle-configuration-effect-on-the-performance-o
CU Author: Christophe Bastien
Date: Sept 2017


Publication Name: Nonlinear Optimization of Vehicle Safety Structures: Modeling of Structures Subjected to Large Deformations
Publication URL: 

Publication Name: Road Safety Trust (RST 65-3-2017) “Reducing Road Traffic Casualties through Improved Forensic Techniques and Vehicle Design (“RoaD”) - Final Report
Publication URL: https://pureportal.coventry.ac.uk/en/publications/road-safety-trust-rst-65-3-2017-reducing-road-traffic-casualties-
CU Author: Christophe Bastien
Date: 22 May 2020


Publication Name: Vehicle Seat Assembly
Publication URL: https://pureportal.coventry.ac.uk/en/publications/vehicle-seat-assembly-2
CU Author: Christophe Bastien
Date: 8 Jan 2020


Publication Name: A Minimum Area Discrepancy Method (MADM) for Force Displacement Response Correlation
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-minimum-area-discrepancy-method-madm-for-force-displacement-res
CU Author: Christophe Bastien
Date: 18 Aug 2019


Publication Name: Deployment of HyperWorks for Undergraduate Teaching
Publication URL: https://pureportal.coventry.ac.uk/en/publications/deployment-of-hyperworks-for-undergraduate-teach

Publication Name: A narrative from the right to the city in Iran: the theoretical–practical continuum of urban development in Bandar Abbas
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-narrative-from-the-right-to-the-city-in-iran-the-theoreticalpra
CU Author: Hossein Sadri
Date: 7 Feb 2023


Publication Name: A New Approach in Urban Development Based on the Right to the City Approach
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-new-approach-in-urban-development-based-on-the-right-to-the-cit
CU Author: Hossein Sadri
Date: Jun 2022


Publication Name: Ceci N’est Pas Un Citron: Habitat Vs Architecture
Publication URL: https://pureportal.coventry.ac.uk/en/publications/ceci-nest-pas-un-citron-habitat-vs-architecture
CU Author: Hossein Sadri
Date: 1 Jul 2021


Publication Name: Cities without Capitalism
Publication URL: https://pureportal.coventry.ac.uk/en/publications/cities-without-capitalism
CU Author: Hossein Sadri
Date: 23 Jul 2021


Public